# Laboratory 1.2

Welcome to Lab 1.2. In this lab, we will learn about the Canny edge detection algorithm, the processing steps involved, and how to apply Canny edge detection to real-world images and videos.

## Instructions

Below is a detailed guide on how to program and apply the Canny edge detection algorithm in practice.


### Libraries



In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

### Reading Images


In [ ]:
image_path = [...]
# Read the image and return a 3-dimensional matrix corresponding to the color channels B,G,R
img = cv2.imread(image_path)

# Display the image in a window named "Image"
cv2.imshow("Image",img)

# Wait for the user to press any key to close the window
cv2.waitKey(0)

# Close all windows
cv2.destroyAllWindows()

### Gaussian Blur

#### Solution to exercise 1

In [ ]:
def gaussian_blur(image, kernel_size=5, sigma=1.4):
    return cv2.GaussianBlur(image, (kernel_size, kernel_size), sigma)

In [ ]:
# Load image and convert to grayscale
gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
blurred_image = gaussian_blur(gray_image)

#### Result

In [ ]:
# Create a figure with 2 subplots
fig, axs = plt.subplots(1, 2, figsize=(10, 5))

# Show the first image
axs[0].imshow(gray_image, cmap='gray')
axs[0].axis('off')  # Turn off axis display

# Show the second image
axs[1].imshow(blurred_image, cmap='gray')
axs[1].axis('off')  # Turn off axis display

# Display figure
plt.show()

### Gradient Calculation


#### Solution to exercise 2

In [ ]:
def gradient(image):
    sobelx = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=3)
    sobely = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=3)
    gradient_magnitude = np.hypot(sobelx, sobely)
    gradient_direction = np.arctan2(sobely, sobelx)
    return gradient_magnitude, gradient_direction

In [ ]:
gradient_magnitude, gradient_direction = gradient(blurred_image)

#### Result

In [ ]:
# Create a figure with 2 subplots
fig, axs = plt.subplots(1, 2, figsize=(10, 5))

# Show the first image
axs[0].imshow(blurred_image, cmap='gray')
axs[0].axis('off')  # Turn off axis display

# Show the second image
axs[1].imshow(gradient_magnitude, cmap='gray')
axs[1].axis('off')  # Turn off axis display

# Display figure
plt.show()

### Non-maximum Suppression


#### Solution to exercise 3

In [ ]:
def non_maximum_suppression(gradient_magnitude, gradient_direction):
    M, N = gradient_magnitude.shape
    Z = np.zeros((M, N), dtype=np.float32)
    angle = gradient_direction * 180. / np.pi
    angle[angle < 0] += 180

    for i in range(1, M-1):
        for j in range(1, N-1):
            try:
                q = 255
                r = 255

                ### Your code starts here ###
                # Angle 0
                if (0 <= angle[i, j] < 22.5) or (157.5 <= angle[i, j] <= 180):
                    q = gradient_magnitude[i, j + 1]
                    r = gradient_magnitude[i, j - 1]
                # Angle 45
                elif 22.5 <= angle[i, j] < 67.5:
                    q = gradient_magnitude[i + 1, j - 1]
                    r = gradient_magnitude[i - 1, j + 1]
                # Angle 90
                elif 67.5 <= angle[i, j] < 112.5:
                    q = gradient_magnitude[i + 1, j]
                    r = gradient_magnitude[i - 1, j]
                # Angle 135
                elif 112.5 <= angle[i, j] < 157.5:
                    q = gradient_magnitude[i - 1, j - 1]
                    r = gradient_magnitude[i + 1, j + 1]
                ### Your code ends here ###

                if (gradient_magnitude[i, j] >= q) and (gradient_magnitude[i, j] >= r):
                    Z[i, j] = gradient_magnitude[i, j]
                else:
                    Z[i, j] = 0

            except IndexError as e:
                pass

    return Z

In [ ]:
nms_image = non_maximum_suppression(gradient_magnitude, gradient_direction)

#### Result

In [ ]:
# Create a figure with 2 subplots
fig, axs = plt.subplots(1, 2, figsize=(10, 5))

# Show the first image
axs[0].imshow(gradient_magnitude, cmap='gray')
axs[0].axis('off')  # Turn off axis display

# Show the second image
axs[1].imshow(nms_image, cmap='gray')
axs[1].axis('off')  # Turn off axis display

# Display figure
plt.show()

### Double Threshold


#### Solution to exercise 4


In [ ]:
def double_threshold(image, low_threshold, high_threshold):
    strong = 255
    weak = 50
    strong_i, strong_j = np.where(image >= high_threshold)
    weak_i, weak_j = np.where((image >= low_threshold) & (image < high_threshold))

    result = np.zeros_like(image)
    result[strong_i, strong_j] = strong
    result[weak_i, weak_j] = weak

    return result

In [ ]:
low_threshold = 0.05 * nms_image.max()
high_threshold = 0.15 * nms_image.max()
thresholded_image = double_threshold(nms_image, low_threshold, high_threshold)

#### Result

In [ ]:
# Create a figure with 2 subplots
fig, axs = plt.subplots(1, 2, figsize=(10, 5))

# Show the first image
axs[0].imshow(nms_image, cmap='gray')
axs[0].axis('off')  # Turn off axis display

# Show the second image
axs[1].imshow(thresholded_image, cmap='gray')
axs[1].axis('off')  # Turn off axis display

# Display figure
plt.show()

### Edge Tracking by Hysteresis


#### Solution to exercise 5


In [ ]:
def edge_tracking(image, weak=50, strong=255):
    M, N = image.shape
    for i in range(1, M-1):
        for j in range(1, N-1):
            ### Your code starts here ###
            if (image[i, j] == weak):
                if ((image[i+1, j-1] == strong) or (image[i+1, j] == strong) or (image[i+1, j+1] == strong)
                    or (image[i, j-1] == strong) or (image[i, j+1] == strong)
                    or (image[i-1, j-1] == strong) or (image[i-1, j] == strong) or (image[i-1, j+1] == strong)):
                    image[i, j] = strong
                else:
                    image[i, j] = 0
            ### Your code ends here ###
    return image

In [ ]:
final_image = edge_tracking(thresholded_image)

#### Result

In [ ]:
# Create a figure with 2 subplots
fig, axs = plt.subplots(1, 2, figsize=(10, 5))

# Show the first image
axs[0].imshow(thresholded_image, cmap='gray')
axs[0].axis('off')  # Turn off axis display

# Show the second image
axs[1].imshow(final_image, cmap='gray')
axs[1].axis('off')  # Turn off axis display

# Display figure
plt.show()

### Combining All Steps



#### Solution to exercise 6


In [ ]:
def canny_edge_detection(image, low_threshold_ratio=0.05, high_threshold_ratio=0.15, sigma=1.4):

    # Blur the image by applying a Gaussian filter
    blurred_image = gaussian_blur(image, sigma=sigma)

    # Calculate Gradient
    gradient_magnitude, gradient_direction = gradient(blurred_image)

    # Non-maximum Suppression
    nms_image = non_maximum_suppression(gradient_magnitude, gradient_direction)

    # Select threshold
    low_threshold = low_threshold_ratio * nms_image.max()
    high_threshold = high_threshold_ratio * nms_image.max()

    # Edge tracking
    thresholded_image = double_threshold(nms_image, low_threshold, high_threshold)
    final_image = edge_tracking(thresholded_image)
    return final_image

In [ ]:
# Load image and convert to grayscale
image = cv2.imread('/road_2.jpg', cv2.IMREAD_GRAYSCALE)
edges = canny_edge_detection(image)

#### Result

In [ ]:
# Create a figure with 2 subplots
fig, axs = plt.subplots(1, 2, figsize=(10, 5))

# Show the first image
axs[0].imshow(image, cmap='gray')
axs[0].axis('off')  # Turn off axis display

# Show the second image
axs[1].imshow(edges, cmap='gray')
axs[1].axis('off')  # Turn off axis display

# Display figure
plt.show()

### Using the Built-in Function in the OpenCV Library


#### Solution to exercise 7


In [ ]:
# Read and convert image to grayscale
image = cv2.imread([...], cv2.IMREAD_GRAYSCALE)
canny_img = cv2.Canny(image, 150, 450)

#### Result

In [ ]:
# Create a figure with 2 subplots
fig, axs = plt.subplots(1, 2, figsize=(10, 5))

# Show the first image
axs[0].imshow(image, cmap='gray')
axs[0].axis('off')  # Turn off axis display

# Show the second image
axs[1].imshow(canny_img, cmap='gray')
axs[1].axis('off')  # Turn off axis display

# Display figure
plt.show()

### Applying Canny to Video

#### Solution to exercise 8

In [ ]:
# Path to input video and output video
input_video_path = [...]
output_video_path = [...]

# Open input video
cap = cv2.VideoCapture(input_video_path)

# Get information about the format of the input video
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')

# Create a VideoWriter object to write the output video
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height), isColor=False)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert frame to grayscale
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Method 1: Apply self-coded Canny edge detection filter
    # edges = canny_edge_detection(gray_frame)

     # Method 2: Apply the Canny edge detection filter available in the library
    edges = cv2.Canny(gray_frame, 150, 450)

    # Record processed frames to output video
    out.write(edges)

# Free up resources
cap.release()
out.release()
cv2.destroyAllWindows()
